In [0]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [2]:
mnist = tf.keras.datasets.mnist

(xtrain, ytrain), (xtest, ytest) = mnist.load_data()
xtrain, xtest = xtrain / 255.0, xtest / 255.0

# Add a channels dimension
xtrain = xtrain[..., tf.newaxis]
xtest = xtest[..., tf.newaxis]

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
trainds = tf.data.Dataset.from_tensor_slices(
    (xtrain, ytrain)).shuffle(10000).batch(32)

testds = tf.data.Dataset.from_tensor_slices((xtest, ytest)).batch(32)

In [0]:
class Model1(Model):
  def __init__(self):
    super(Model1, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

mm = Model1()

In [0]:
lossobject = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [0]:
trainloss = tf.keras.metrics.Mean(name='trainloss')
trainaccuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='trainaccuracy')

testloss = tf.keras.metrics.Mean(name='testloss')
testaccuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='testaccuracy')

In [0]:
@tf.function
def trainstep(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = mm(images, training=True)
    loss = lossobject(labels, predictions)
  gradients = tape.gradient(loss, mm.trainable_variables)
  optimizer.apply_gradients(zip(gradients, mm.trainable_variables))

  trainloss(loss)
  trainaccuracy(labels, predictions)

In [0]:
@tf.function
def teststep(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = mm(images, training=False)
  tloss = lossobject(labels, predictions)

  testloss(tloss)
  testaccuracy(labels, predictions)

In [18]:
EPOCHS = 5

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  trainloss.reset_states()
  trainaccuracy.reset_states()
  testloss.reset_states()
  testaccuracy.reset_states()

  for images, labels in trainds:
    trainstep(images, labels)

  for testimages, testlabels in testds:
    teststep(testimages, testlabels)

  temp = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(temp.format(epoch + 1,
                        trainloss.result(),
                        trainaccuracy.result() * 100,
                        testloss.result(),
                        testaccuracy.result() * 100))

Epoch 1, Loss: 0.04461739957332611, Accuracy: 98.61833190917969, Test Loss: 0.0519699864089489, Test Accuracy: 98.25
Epoch 2, Loss: 0.024247992783784866, Accuracy: 99.22500610351562, Test Loss: 0.049404535442590714, Test Accuracy: 98.4000015258789
Epoch 3, Loss: 0.015113381668925285, Accuracy: 99.50167083740234, Test Loss: 0.049856990575790405, Test Accuracy: 98.62999725341797
Epoch 4, Loss: 0.011371432803571224, Accuracy: 99.61833190917969, Test Loss: 0.06258073449134827, Test Accuracy: 98.4000015258789
Epoch 5, Loss: 0.007489749230444431, Accuracy: 99.75666046142578, Test Loss: 0.07448948174715042, Test Accuracy: 98.20999908447266
